In [1]:
from cdag import Graph, load_data, read_table
import re, pandas as pd

In [2]:
tg = pd.read_csv('chipsoft_logic_table_guids.csv', sep=';')
table_name = {}
for elt in tg.to_dict('records'):
    table_name[elt['classguid']] = elt['classid']

In [18]:
def resolve_guid(match):
    return table_name.get(match.group(0), '<unknown>')

In [3]:
iv = pd.read_csv('config_instvars.csv')
iv['VALUE'] = iv['VALUE'].fillna('X""')

In [11]:
# iv.groupby('INSTTYPE').count()
# C: 5671, D: 351, G: 6382, L: 70606, U: 904852
# type L has only a few possibilities for VALUE: LT©, LTTTTTT, CT
# type U has more options for value (including XML strings) but appears to be related to individual user rights or settings
# types D and G always have owner=CHIPSOFT

In [4]:
type_c = iv.loc[iv['INSTTYPE'] == 'C']
# type_c.head(10)

In [13]:
# Unicode dingbats, open and closed circled digits
OD1, CD1 = '\u2780', '\u2776'
OD2, CD2 = '\u2781', '\u2777'
OD3, CD3 = '\u2782', '\u2778'
level1_rx = re.compile(r'"([\u2782\u2778\u2781\u2777\x20\x21\x23-\x7E]+?)"')
level2_rx = re.compile(r'""([\u2782\u2778\x20\x21\x23-\x7E]+?)""')
level3_rx = re.compile(r'""""([\x20\x21\x23-\x7E]+?)""""')

def tokenize(s):
    # TODO: are "" and """" artefacts grep Sof CSV export? 
    if s.startswith('C'):
        s3 = level3_rx.sub('\u2782\g<1>\u2778', s.lstrip('C').rstrip('\x01'))
        s2 = level2_rx.sub('\u2781\g<1>\u2777', s3)
        s1 = level1_rx.sub('\u2780\g<1>\u2776', s2)
        return s1
    else:
        return ''
    
def parse(s):
    if s.startswith('\u2780') and s.endswith('\u2776'):
        # TODO: handle levels 2 and 3
        return s[1:-1].split('\u2776,\u2780')
    else:
        print(f"begin={s[0]} end={s[-1]}")
        return

In [23]:
#occurring, missing = {}, set()
#guid_rx = re.compile(r'\{[A-Z0-9-]+?\}')
#sep = '-'*10
#for k, elt in enumerate(type_c.to_dict('records')):
#    value = guid_rx.sub(resolve_guid, elt['VALUE'])
#    # matches = guid_rx.findall(value)
#    if k < 4:
#        print(f"{sep}\nvalue {k}")
#        tokens = tokenize(value)
#        obj = parse(tokens)
#        for elt in obj:
#            print('\t'+elt)

In [24]:
wc = pd.read_csv('config_workcontext.csv')
wc.set_index('Id', inplace=True)
ws = pd.read_csv('config_wcsegments.csv')
ws.set_index('ConfiguredWorkContextId', inplace=True)

In [25]:
gr = pd.read_csv('ziscon_groepen.csv')

In [26]:
us = pd.read_csv('ziscon_user.csv',
                 dtype={'FUNKTIE':'string', 'BEHEERDER':'boolean', 'WACHTWOORD':'string',
                        'DISABLED': 'boolean', 'UZINR': 'string', 'ROLE': 'string', 'WRONGCHECKS': 'string'})

In [28]:
wcs = wc.join(ws, how='inner')
#result = wcs.loc[~wcs['Disabled']].copy()
#result.to_csv('workcontext.csv')
wcs.loc[(wc['OwnerId']=='ZH0085') & wc['SettingId'].str.contains('AG_US_BEK')]